## Preparing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
from recibrew.nn.lstm_bahdaneu import Encoder

In [4]:
from recibrew.data_util import construct_torchtext_iterator
train_csv = '../data/processed/train.csv'
dev_csv = '../data/processed/dev.csv'
test_csv = '../data/processed/test.csv'
constructed_ttext = construct_torchtext_iterator(train_csv, dev_csv, test_csv, device='cpu', fix_length=None)

In [6]:
train_iter = constructed_ttext['train_iter']

In [9]:
src_field = constructed_ttext['src_field']
max_vocab = len(src_field.vocab)


In [10]:
btch = next(train_iter.__iter__())
src, tgt = btch.src, btch.tgt

In [13]:
enc_gru = Encoder(max_vocab, dropout=None)

In [16]:
enc_gru(src)[0].shape

torch.Size([15, 64, 100])